<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_%D1%87%D0%B0%D1%81%D1%82%D1%8C_2_%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Выполнить практическую работу из лекционного ноутбука.
1. Построить RNN-ячейку на основе полносвязных слоев
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”'''


In [ ]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети

In [ ]:
#Загрузка данных. Будем работать с датасетом реплик из Симпсонов. Нам нужно извлечь предобработанные тексты и закодировать их числами
df = pd.read_csv('/content/drive/MyDrive/Datasets/simpson/simpsons_script_lines.csv')
df.head()

<ipython-input-2-1979033914>:2: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Datasets/simpson/simpsons_script_lines.csv')


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [ ]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:10]

['no actually it was a little of both sometimes when a disease is in all the magazines and all the news shows its only natural that you think you have it',
 'wheres mr bergstrom',
 'i dont know although id sure like to talk to him he didnt touch my lesson plan what did he teach you',
 'that life is worth living',
 'the polls will be open from now until the end of recess now just in case any of you have decided to put any thought into this well have our final statements martin',
 'i dont think theres anything left to say',
 'bart',
 'victory party under the slide',
 nan,
 'mr bergstrom mr bergstrom']

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]
#text

In [ ]:
'''Создаём массив с данными
Нужно
    Разбить данные на токены (у нас символы)
    Закодировать числами
    Превратить в эмбеддинги'''

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # 28 символов, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс
len(INDEX_TO_CHAR)

28

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

X[0:5]

tensor([[19, 25,  3, 13, 16, 20, 26, 13, 11, 11,  9,  3, 27, 20,  3,  5, 13,  2,
          3, 13,  3, 11, 27, 20, 20, 11,  7,  3, 25, 21,  3, 14, 25, 20, 15,  3,
          2, 25, 17,  7, 20, 27, 17,  7,  2,  3,  5, 15,  7, 19],
        [ 5, 15,  7, 10,  7,  2,  3, 17, 10,  3, 14,  7, 10, 24,  2, 20, 10, 25,
         17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [27,  3,  8, 25, 19, 20,  3,  4, 19, 25,  5,  3, 13, 11, 20, 15, 25, 26,
         24, 15,  3, 27,  8,  3,  2, 26, 10,  7,  3, 11, 27,  4,  7,  3, 20, 25,
          3, 20, 13, 11,  4,  3, 20, 25,  3, 15, 27, 17,  3, 15],
        [20, 15, 13, 20,  3, 11, 27, 21,  7,  3, 27,  2,  3,  5, 25, 10, 20, 15,
          3, 11, 27, 12, 27, 19, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [20, 15,  7,  3,  6, 25, 11, 11,  2,  3,  5, 27, 11, 11,  3, 14,  7,  3,
       

In [ ]:
#Embedding и RNN ячейки.Каждому токену мы хотим сопоставить не просто число, но вектор. Поэтому вектор текста нам нужно умножить на матрицу эмбеддингов,
#которая тоже будет учиться в процессе обучения нейросети. Для создания такой матрицы нам нужен слой nn.Embedding

embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)  # размер словаря * размер вектора для кодировки каждого слова
t = embeddings(X[0:5])
t.shape, X[0:5].shape

(torch.Size([5, 50, 28]), torch.Size([5, 50]))

In [ ]:
rnn = torch.nn.RNN(28, 128, batch_first=True)  # на вход - размер эмбеддинга, размер скрытого состояния и порядок размерностей
o, s = rnn(t)
# вектора для слов: батч * число токенов * размер скрытого состояния
# вектор скрытого состояния: число вектров (один) * батч * размер скрытого состояния
o.shape, s.shape

(torch.Size([5, 50, 128]), torch.Size([1, 5, 128]))

In [ ]:
'''Реализация сети с RNN
3 слоя:
    Embeding (30)
    RNN (hidden_dim=128)
    Полносвязный слой для предсказания буквы (28, то есть размер словаря)'''

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)
model = Network()

criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
#Обучение

for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))


Epoch 0. Time: 51.196, Train loss: 1.709
Epoch 1. Time: 50.968, Train loss: 1.700
Epoch 2. Time: 51.730, Train loss: 1.691
Epoch 3. Time: 56.287, Train loss: 1.686
Epoch 4. Time: 55.567, Train loss: 1.683
Epoch 5. Time: 51.585, Train loss: 1.678
Epoch 6. Time: 52.313, Train loss: 1.675
Epoch 7. Time: 52.219, Train loss: 1.673
Epoch 8. Time: 50.980, Train loss: 1.673
Epoch 9. Time: 51.636, Train loss: 1.671


In [ ]:
#Генерация текста. Сначала отправлем в модель буквы из предложения (прогревая состояние). Затем берём самую вероятную букву и добавляем её в предложение. Повторяем пока не получим none (0)

In [ ]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [ ]:
CHAR_TO_INDEX['none']

0